## Import Libraries

In [3]:
import os
from crewai import Agent, Task, Crew, Sequential, Parallel
from langchain.tools import PythonREPLTool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.llms import HuggingFaceHub


ImportError: cannot import name 'Sequential' from 'crewai' (/opt/anaconda3/lib/python3.12/site-packages/crewai/__init__.py)

## Set your HuggingFace token

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your-hf-token-here"


## Load the Mistral LLM


In [ ]:
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.5, "max_new_tokens": 512}
)

## Load Tools

In [ ]:
search_tool = DuckDuckGoSearchRun()
python_tool = PythonREPLTool()

## Agent Definitions



In [ ]:
loader_agent = Agent(
    role="Loader Agent",
    goal="Search and retrieve electric consumption data for Indian states (2018-2023).",
    backstory="A research assistant who finds credible data sources from the web.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)
#This agent finds dataset links online.

In [ ]:
analyzer_agent = Agent(
    role="Analyzer Agent",
    goal="Analyze the CSV to find top 3 states with highest average consumption.",
    backstory="A data analyst who uses pandas to extract patterns.",
    tools=[python_tool],
    llm=llm,
    verbose=True
)
#This one loads CSV and analyzes with pandas.

In [ ]:
visualizer_agent = Agent(
    role="Visualizer Agent",
    goal="Create bar charts to show electricity consumption per year by state.",
    backstory="A matplotlib expert for Indian state electricity data.",
    tools=[python_tool],
    llm=llm,
    verbose=True
)
#Makes visual plots of electricity data.

In [ ]:
summarizer_agent = Agent(
    role="Summarizer Agent",
    goal="Write a 2-3 sentence summary about which states consumed the most electricity and why.",
    backstory="A professional executive summary writer.",
    tools=[],
    llm=llm,
    verbose=True
)
#Pure LLM-based summary writer.



## Task Definitions

In [ ]:
load_task = Task(
    description="Search for electricity consumption datasets for Indian states (2018–2023). Focus on government sources. Print top 3 links.",
    agent=loader_agent
)

analyze_task = Task(
    description="Load a CSV file, calculate average consumption, and print top 3 states.",
    agent=analyzer_agent
)

visualize_task = Task(
    description="Use matplotlib/seaborn to plot electricity consumption for each state.",
    agent=visualizer_agent
)

summarize_task = Task(
    description="Write a summary of insights from the data and visualizations in 2–3 sentences.",
    agent=summarizer_agent
)
#Each task explains what the agent should do.

## Flow Definition

In [ ]:
flow = Sequential(
    steps=[
        load_task,
        Parallel(steps=[analyze_task, visualize_task, summarize_task])
    ]
)
#First, data is loaded. Then 3 tasks run in parallel.

##  Run and Show Result

In [ ]:
crew = Crew(flow=flow, verbose=True)
result = crew.kickoff()

print("\n Final Result:ssssss
